<a href="https://colab.research.google.com/github/guptatushar2000/Trolling-Aggression-and-Misogyny-Identification/blob/master/Final_TRAC_(Misogyny).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATA ANALYSIS

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import pandas as pd
import numpy as np
import os

import nltk
nltk.download('stopwords')

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from nltk.corpus import stopwords
from nltk.util import ngrams

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
import gensim
import string

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
tweet = pd.read_csv('/gdrive/My Drive/eng/eng/trac2_eng_train.csv')
test = pd.read_csv('/gdrive/My Drive/trac2_eng_dev.csv')

KeyboardInterrupt: ignored

In [0]:
print('There are {} rows and {} columns in train'.format(tweet.shape[0],tweet.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

In [0]:
tweet.head(10)

In [0]:
tweet["Sub-task B"] = tweet["Sub-task B"].map({"GEN": 1, "NGEN": 0}).astype('int64')
test["Sub-task B"] = test["Sub-task B"].map({"GEN": 1, "NGEN": 0}).astype('int64')

In [0]:
Miso_len = tweet[tweet['Sub-task B'] == 1].shape[0]
Not_len = tweet[tweet['Sub-task B'] == 0].shape[0]

In [0]:
plt.rcParams['figure.figsize'] = (7, 5)
plt.bar(10,Miso_len,3, label="Miso", color='red')
plt.bar(15,Not_len,3, label="Not", color='blue')
plt.legend()
plt.ylabel('Number of examples')
plt.title('Propertion of examples')
plt.show()

In [0]:
def length(text):    
    return len(text)

In [0]:
tweet['length'] = tweet['Text'].apply(length)

In [0]:
plt.rcParams['figure.figsize'] = (18.0, 6.0)
bins = 150
plt.hist(tweet[tweet['Sub-task B'] == 0]['length'], alpha = 0.6, bins=bins, label='Not')
plt.hist(tweet[tweet['Sub-task B'] == 1]['length'], alpha = 0.8, bins=bins, label='Miso')
plt.xlabel('length')
plt.ylabel('numbers')
plt.legend(loc='upper right')
plt.xlim(0,150)
plt.grid()
plt.show()

In [0]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=tweet[tweet['Sub-task B']==1]['Text'].str.len()
ax1.hist(tweet_len,color='red')
ax1.set_title('miso tweets')
tweet_len=tweet[tweet['Sub-task B']==0]['Text'].str.len()
ax2.hist(tweet_len,color='Blue')
ax2.set_title('Not miso tweets')
fig.suptitle('Characters in tweets')
plt.show()

In [0]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=tweet[tweet['Sub-task B']==1]['Text'].str.split().map(lambda x: len(x))
ax1.hist(tweet_len,color='red')
ax1.set_title('miso tweets')
tweet_len=tweet[tweet['Sub-task B']==0]['Text'].str.split().map(lambda x: len(x))
ax2.hist(tweet_len,color='blue')
ax2.set_title('Not miso tweets')
fig.suptitle('Words in a tweet')
plt.show()

In [0]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
word=tweet[tweet['Sub-task B']==1]['Text'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax1,color='red')
ax1.set_title('miso')
word=tweet[tweet['Sub-task B']==0]['Text'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax2,color='blue')
ax2.set_title('Not miso')
fig.suptitle('Average word length in each tweet')

In [0]:
def create_corpus(target):
    corpus=[]
    
    for x in tweet[tweet['Sub-task B']==target]['Text'].str.split():
        for i in x:
            corpus.append(i)
    return corpus

In [0]:
def create_corpus_df(tweet, target):
    corpus=[]
    
    for x in tweet[tweet['Sub-task B']==target]['Text'].str.split():
        for i in x:
            corpus.append(i)
    return corpus

In [0]:
corpus=create_corpus(0)

dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1
        
top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10]

In [0]:
np.array(stop)

In [0]:
plt.rcParams['figure.figsize'] = (18.0, 6.0)
x,y=zip(*top)
plt.bar(x,y)

In [0]:
corpus=create_corpus(1)

dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1

top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10] 
    

plt.rcParams['figure.figsize'] = (18.0, 6.0)
x,y=zip(*top)
plt.bar(x,y)

In [0]:
plt.figure(figsize=(16,5))
corpus=create_corpus(1)

dic=defaultdict(int)
special = string.punctuation
for i in (corpus):
    if i in special:
        dic[i]+=1
        
x,y=zip(*dic.items())
plt.bar(x,y)

In [0]:
plt.figure(figsize=(16,5))
corpus=create_corpus(0)
dic=defaultdict(int)
special = string.punctuation
for i in (corpus):
    if i in special:
        dic[i]+=1
        
x,y=zip(*dic.items())
plt.bar(x,y,color='green')

In [0]:
counter=Counter(corpus)
most=counter.most_common()
x=[]
y=[]
for word,count in most[:40]:
    if (word not in stop) :
        x.append(word)
        y.append(count)

In [0]:
sns.barplot(x=y,y=x)

In [0]:
def get_top_tweet_bigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [0]:
plt.figure(figsize=(16,5))
top_tweet_bigrams=get_top_tweet_bigrams(tweet['Text'])[:10]
x,y=map(list,zip(*top_tweet_bigrams))
sns.barplot(x=y,y=x)

In [0]:
df=pd.concat([tweet,test])
df.shape

In [0]:
example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"

In [0]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

remove_URL(example)

In [0]:
df['Text']=df['Text'].apply(lambda x : remove_URL(x))

In [0]:
example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""

In [0]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
print(remove_html(example))

In [0]:
df['Text']=df['Text'].apply(lambda x : remove_html(x))

In [0]:
FLAGS = re .MULTILINE | re .DOTALL 
def tokenize ( text ): # Different regex parts for smiley faces 
  eyes = r"[8:=;]" 
  nose = r"[’‘\−]?" 
  # function 
  def re_sub ( pattern , repl ): 
    return re.sub( pattern , repl , text , flags=FLAGS) 
  text = re_sub(r" https ?:\/\/\S+\b|www\.(\w+\.)+\S∗" , " url ") 
  # Smile −− :) , : ) , :−), (: , ( : , (−:, : ’) 
  text = re_sub(r"(:\s ?\)|: −\)|\(\s ?:|\( −:|:\ ’\))" , " em_smile ") 
  # Laugh −− :D, : D, :−D, xD, x−D, XD, X−D 
  text = re_sub(r"(:\s?D|:−D|x−?D|X−?D)" , " em_laugh ") 
  # Love −− <3, :∗ 
  text = re_sub(r"( <3|:.)" , " em_love ") 
  # Wink −− ;−), ;) , ;−D, ;D, (; , (−; 
  text = re_sub(r"(;−?\)|;−?D|\(−?;)" , " em_wink ") 
  # Sad −− :−(, : ( , :( , ): , )−: 
  text = re_sub(r"(:\ s ?\(|: −\(|\)\s ?:|\) −:)" , " em_sad ") 
  # Cry −− : ,( , : ’( , :"( 
  text = re_sub(r"(: ,\(|:\ ’\(|:”\()" , " em_cry ")
  # remove funnnnny −−> funny 
  text = re_sub(r"(.)\1+" , r"\1\1") 
  # remove & 
  text = re_sub(r"(−|\’)" , "") 
  text = re_sub(r"@[0−9]+−", " number ") 
  text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}". format (eyes , nose , nose , eyes ) , " em_positive ") 
  text = re_sub(r"{}{}p+". format (eyes , nose ) , " em_positive ") 
  text = re_sub(r"{}{}\(+|\)+{}{}". format (eyes , nose , nose , eyes ) , " em_negative ") 
  text = re_sub(r"{}{}[\/| l ∗]". format (eyes , nose ) , " em_neutralface ") 
  text = re_sub(r"−", r" " ) 
  text = re_sub(r"([ pls?s ]){2 ,}" , r"\1") 
  text = re_sub(r"([ plz?z]){2 ,}" , r"\1") 
  text = re_sub(r"\\n" , r" ") 
  text = re_sub(r" sx " ," sex ") 
  text = re_sub(r" u " ," you ") 
  text = re_sub(r" r " ," are ") 
  text = re_sub(r" y " ," why ") 
  text = re_sub(r" Y " ," WHY ") 
  text = re_sub(r"Y " ," WHY ") 
  text = re_sub(r" hv " ," have ") 
  text = re_sub(r" c " ," see ") 
  text = re_sub(r" bcz " ," because ")
  text = re_sub(r" coz " ," because ") 
  text = re_sub(r" v " ," we ") 
  text = re_sub(r" ppl " ," people ") 
  text = re_sub(r" pepl " ," people ") 
  text = re_sub(r" r b i " ," rbi ") 
  text = re_sub(r" R B I " ," RBI ") 
  text = re_sub(r" R b i " ," rbi ") 
  text = re_sub(r" R " ," ARE ") 
  text = re_sub(r" hav " ," have ") 
  text = re_sub(r"R " ," ARE ") 
  text = re_sub(r" U " ," you ") 
  text = re_sub(r"U " ," you ") 
  text = re_sub(r" pls " ," please ") 
  text = re_sub(r"Pls " ," Please ") 
  text = re_sub(r"plz " ," please ") 
  text = re_sub(r"Plz " ," Please ") 
  text = re_sub(r"PLZ " ," Please ") 
  text = re_sub(r"Pls " ," Please ") 
  text = re_sub(r"plz " ," please ") 
  text = re_sub(r"Plz " ," Please ") 
  text = re_sub(r"PLZ" ," Please ") 
  text = re_sub(r" thankz " ," thanks ") 
  text = re_sub(r" thnx " ," thanks ") 
  text = re_sub(r"fuck\w+ " ," fuck ") 
  text = re_sub(r"f.. " ," fuck ") 
  text = re_sub(r"...k " ," fuck ") 
  text = re_sub(r"F.. " ," fuck ") 
  text = re_sub(r"mo..... " ," fucker ") 
  text = re_sub(r"b.... " ," blody ") 
  text = re_sub(r" mc " ," fucker ") 
  text = re_sub(r" MC " ," fucker ") 
  text = re_sub(r" wtf " ," fuck ") 
  text = re_sub(r" ch.t+i*[iy]a " ," fucker ") 
  # text = re_sub(r" ch.tya " ," fucker ") 
  # text = re_sub(r" ch..tia " ," fucker ") 
  text = re_sub(r" C...yas " ," fucker ") 
  text = re_sub(r"l.... " ," shit ") 
  text = re_sub(r" As+....S" ," ASSHOLES") 
  text = re_sub(r" di....s" ," fucker ") 
  text = re_sub(r" nd " ," and ") 
  text = re_sub(r"Nd " ,"and ") 
  text = re_sub(r"( ind [vs]pak )" , " india versus pakistan ") 
  text = re_sub(r"(pak[vs] ind )" , " pakistan versus india ") 
  text = re_sub(r"( indvsuae )" , " india versus United Arab Emirates ") 
  text = re_sub(r"[sS] hut [Dd]own[jnuJNU]" , " shut down jnu ") 
  return text

In [0]:
df['Text'] = df['Text'].apply(tokenize)

In [0]:
df.head()

In [0]:
!pip3 install emoji --upgrade

In [0]:
import emoji
def demojize(text):
  return emoji.demojize(text)

In [0]:
df.head()

In [0]:
def remove_punct(text):
    table=str.maketrans(string.punctuation, ' '*len(string.punctuation))
    return text.translate(table)

example="I am a #king"
print(remove_punct(example))

In [0]:
df['Text']=df['Text'].apply(lambda x : remove_punct(x))

In [0]:
df['Text'] = df['Text'].apply(demojize)

In [0]:
corpus_new1=create_corpus_df(df,1)
len(corpus_new1)

In [0]:
corpus_new1[:10]

In [0]:
# Generating the wordcloud with the values under the category dataframe
plt.figure(figsize=(12,8))
word_cloud = WordCloud(
                          background_color='black',
                          max_font_size = 80
                         ).generate(" ".join(corpus_new1[:50]))
plt.imshow(word_cloud)
plt.axis('off')
plt.show()

In [0]:
corpus_new0=create_corpus_df(df,0)
len(corpus_new0)

In [0]:
corpus_new0[:10]

In [0]:
# Generating the wordcloud with the values under the category dataframe
plt.figure(figsize=(12,8))
word_cloud = WordCloud(
                          background_color='black',
                          max_font_size = 80
                         ).generate(" ".join(corpus_new0[:50]))
plt.imshow(word_cloud)
plt.axis('off')
plt.show()

In [0]:
df.head(10)

In [0]:
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = df["Text"].tolist()
list_labels = df["Sub-task B"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=40)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [0]:
def plot_LSA(test_data, test_labels, savepath="PCA_demo.csv", plot=True):
        lsa = TruncatedSVD(n_components=2)
        lsa.fit(test_data)
        lsa_scores = lsa.transform(test_data)
        color_mapper = {label:idx for idx,label in enumerate(set(test_labels))}
        color_column = [color_mapper[label] for label in test_labels]
        colors = ['yellow','red']
        if plot:
            plt.scatter(lsa_scores[:,0], lsa_scores[:,1], s=8, alpha=.8, c=test_labels, cmap=matplotlib.colors.ListedColormap(colors))
            orange_patch = mpatches.Patch(color='yellow', label='Not')
            blue_patch = mpatches.Patch(color='red', label='Miso')
            plt.legend(handles=[orange_patch, blue_patch], prop={'size': 30})

fig = plt.figure(figsize=(16, 16))          
plot_LSA(X_train_counts, y_train)
plt.show()

In [0]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [0]:
plot_LSA(X_train_tfidf, y_train)
plt.show()

PRE-PROCESSING OF DATA

In [0]:
from sklearn import preprocessing

In [0]:
train_data = pd.read_csv('/gdrive/My Drive/eng/eng/trac2_eng_train.csv')
test_data = pd.read_csv('/gdrive/My Drive/trac2_eng_dev.csv')

In [0]:
train_data['Text']=train_data['Text'].str.lower()
train_data['Text']=train_data['Text'].apply(lambda x : remove_URL(x))
train_data['Text']=train_data['Text'].apply(lambda x : remove_html(x))
train_data['Text']=train_data['Text'].apply(lambda x : tokenize(x))
train_data['Text']=train_data['Text'].apply(lambda x : demojize(x))
train_data['Text']=train_data['Text'].apply(lambda x : remove_punct(x))

In [0]:
train_data.head()

In [0]:
!pip3 install pyspellchecker

In [0]:
train_data.drop(['Sub-task A'], axis=1, inplace=True)

In [0]:
train_data["Sub-task B"] = train_data["Sub-task B"].map({"GEN": 1, "NGEN": 0})
test_data["Sub-task B"] = test_data["Sub-task B"].map({"GEN": 1, "NGEN": 0})

In [0]:
train_data.head()

In [0]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler

In [0]:
ros = RandomOverSampler(random_state=777)
train_data['Text'], train_data['Sub-task B'] = ros.fit_resample(train_data['Text'], train_data['Sub-task B'])

In [0]:
nltk.download('stopwords') 
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [0]:
def stopwords(text):
      return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [0]:
train_data["Text"] = train_data["Text"].apply(stopwords)

In [0]:
from collections import Counter
cnt = Counter()
for text in train_data["Text"].values:
    for word in text.split():
        cnt[word] +=1
        
cnt.most_common(50) 

In [0]:
freq = set([w for (w, wc) in cnt.most_common(10)])
def freqwords(text):
    return " ".join([word for word in str(text).split() if word not 
in freq])
train_data["Text"] = train_data["Text"].apply(freqwords)
train_data["Text"].head()

In [0]:
test_data['Text']=test_data['Text'].str.lower()
test_data['Text']=test_data['Text'].apply(lambda x : remove_URL(x))
test_data['Text']=test_data['Text'].apply(lambda x : remove_html(x))
test_data['Text']=test_data['Text'].apply(lambda x : tokenize(x))
test_data['Text']=test_data['Text'].apply(lambda x : demojize(x))
test_data['Text']=test_data['Text'].apply(lambda x : remove_punct(x))
test_data["Text"] = test_data["Text"].apply(stopwords)
test_data["Text"] = test_data["Text"].apply(freqwords)

BERT IMPLEMENTATION

In [0]:
import sys
sys.version

In [0]:
!pip3 install fastai

In [0]:
%%bash
pip install transformers

In [0]:
from pathlib import Path 

import os

import torch
import torch.optim as optim

import random 

# fastai
from fastai import *
from fastai.text import *
from fastai.callbacks import *

# transformers
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig

In [0]:
import fastai
import transformers
print('fastai version :', fastai.__version__)
print('transformers version :', transformers.__version__)

In [0]:
MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)
}

In [0]:
# Parameters
seed = 42
use_fp16 = False
bs = 16

model_type = 'bert'
pretrained_model_name='bert-base-uncased'

In [0]:
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]

In [0]:
model_class.pretrained_model_archive_map.keys()

In [0]:
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [0]:
seed_all(seed)

In [0]:
class TransformersBaseTokenizer(BaseTokenizer):
    """Wrapper around PreTrainedTokenizer to be compatible with fast.ai"""
    def __init__(self, pretrained_tokenizer: PreTrainedTokenizer, model_type = 'bert', **kwargs):
        self._pretrained_tokenizer = pretrained_tokenizer
        self.max_seq_len = pretrained_tokenizer.max_len
        self.model_type = model_type

    def __call__(self, *args, **kwargs): 
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length and add the spesial tokens"""
        CLS = self._pretrained_tokenizer.cls_token
        SEP = self._pretrained_tokenizer.sep_token
        if self.model_type in ['roberta']:
            tokens = self._pretrained_tokenizer.tokenize(t, add_prefix_space=True)[:self.max_seq_len - 2]
            tokens = [CLS] + tokens + [SEP]
        else:
            tokens = self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2]
            if self.model_type in ['xlnet']:
                tokens = tokens + [SEP] +  [CLS]
            else:
                tokens = [CLS] + tokens + [SEP]
        return tokens

In [0]:
transformer_tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)
transformer_base_tokenizer = TransformersBaseTokenizer(pretrained_tokenizer = transformer_tokenizer, model_type = model_type)
fastai_tokenizer = Tokenizer(tok_func = transformer_base_tokenizer, pre_rules=[], post_rules=[])

In [0]:
class TransformersVocab(Vocab):
    def __init__(self, tokenizer: PreTrainedTokenizer):
        super(TransformersVocab, self).__init__(itos = [])
        self.tokenizer = tokenizer
    
    def numericalize(self, t:Collection[str]) -> List[int]:
        "Convert a list of tokens `t` to their ids."
        return self.tokenizer.convert_tokens_to_ids(t)
        #return self.tokenizer.encode(t)

    def textify(self, nums:Collection[int], sep=' ') -> List[str]:
        "Convert a list of `nums` to their tokens."
        nums = np.array(nums).tolist()
        return sep.join(self.tokenizer.convert_ids_to_tokens(nums)) if sep is not None else self.tokenizer.convert_ids_to_tokens(nums)
    
    def __getstate__(self):
        return {'itos':self.itos, 'tokenizer':self.tokenizer}

    def __setstate__(self, state:dict):
        self.itos = state['itos']
        self.tokenizer = state['tokenizer']
        self.stoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.itos)})

In [0]:
transformer_vocab =  TransformersVocab(tokenizer = transformer_tokenizer)
numericalize_processor = NumericalizeProcessor(vocab=transformer_vocab)

tokenize_processor = TokenizeProcessor(tokenizer=fastai_tokenizer, include_bos=False, include_eos=False)

transformer_processor = [tokenize_processor, numericalize_processor]

In [0]:
pad_first = bool(model_type in ['bert'])
pad_idx = transformer_tokenizer.pad_token_id

In [0]:
tokens = transformer_tokenizer.tokenize('Salut c est moi, Hello it s me')
print(tokens)
ids = transformer_tokenizer.convert_tokens_to_ids(tokens)
print(ids)
transformer_tokenizer.convert_ids_to_tokens(ids)

In [0]:
databunch = (TextList.from_df(train_data, cols='Text', processor=transformer_processor)
             .split_by_rand_pct(0.1,seed=seed)
             .label_from_df(cols= 'Sub-task B')
             .add_test(test_data)
             .databunch(bs=bs, pad_first=pad_first, pad_idx=pad_idx))

In [0]:
print('[CLS] token :', transformer_tokenizer.cls_token)
print('[SEP] token :', transformer_tokenizer.sep_token)
print('[PAD] token :', transformer_tokenizer.pad_token)
databunch.show_batch()

In [0]:
print('[CLS] id :', transformer_tokenizer.cls_token_id)
print('[SEP] id :', transformer_tokenizer.sep_token_id)
print('[PAD] id :', pad_idx)
test_one_batch = databunch.one_batch()[0]
print('Batch shape : ',test_one_batch.shape)
print(test_one_batch)

In [0]:
# defining our model architecture 
class CustomTransformerModel(nn.Module):
    def __init__(self, transformer_model: PreTrainedModel):
        super(CustomTransformerModel,self).__init__()
        self.transformer = transformer_model
        
    def forward(self, input_ids, attention_mask=None):
        
        #attention_mask = (input_ids!=1).type(input_ids.type()) # Test attention_mask for RoBERTa
        
        logits = self.transformer(input_ids,
                                attention_mask = attention_mask)[0]   
        return logits

In [0]:
config = config_class.from_pretrained(pretrained_model_name)
config.num_labels = 5
config.use_bfloat16 = use_fp16
print(config)

In [0]:
transformer_model = model_class.from_pretrained(pretrained_model_name, config = config)
# transformer_model = model_class.from_pretrained(pretrained_model_name, num_labels = 5)

custom_transformer_model = CustomTransformerModel(transformer_model = transformer_model)

In [0]:
from transformers import AdamW
from functools import partial

CustomAdamW = partial(AdamW, correct_bias=False)

learner = Learner(databunch, 
                  custom_transformer_model, 
                  opt_func = CustomAdamW, 
                  metrics=[accuracy, error_rate, FBeta(beta=1, average='macro')])

# Show graph of learner stats and metrics after each epoch.
learner.callbacks.append(ShowGraph(learner))

# Put learn in FP16 precision mode. --> Seems to not working
if use_fp16: learner = learner.to_fp16()

In [0]:
print(learner.model)

In [0]:
list_layers = [learner.model.transformer.bert.embeddings,
              learner.model.transformer.bert.encoder.layer[0],
              learner.model.transformer.bert.encoder.layer[1],
              learner.model.transformer.bert.encoder.layer[2],
              learner.model.transformer.bert.encoder.layer[3],
              learner.model.transformer.bert.encoder.layer[4],
              learner.model.transformer.bert.encoder.layer[5],
              learner.model.transformer.bert.encoder.layer[6],
              learner.model.transformer.bert.encoder.layer[7],
              learner.model.transformer.bert.encoder.layer[8],
              learner.model.transformer.bert.encoder.layer[9],
              learner.model.transformer.bert.encoder.layer[10],
              learner.model.transformer.bert.encoder.layer[11],
              learner.model.transformer.bert.pooler]

In [0]:
learner.split(list_layers)
num_groups = len(learner.layer_groups)
print('Learner split in',num_groups,'groups')
print(learner.layer_groups)

In [0]:
learner.model_dir='/gdrive/My Drive/eng/eng/'

In [0]:
learner.save('untrain_bert')

In [0]:
seed_all(seed)
learner.load('untrain_bert');

In [0]:
learner.freeze_to(-1)

In [0]:
learner.summary()

In [0]:
learner.lr_find()

In [0]:
learner.recorder.plot(skip_end=10,suggestion=True)

In [0]:
learner.fit_one_cycle(10,max_lr=1e-03,moms=(0.8,0.7))

In [0]:
learner.save('first_cycle_bert')

In [0]:
seed_all(seed)
learner.load('first_cycle_bert');

In [0]:
learner.freeze_to(-2)

In [0]:
lr = 6e-05

In [0]:
learner.fit_one_cycle(10, max_lr=slice(lr*0.95**num_groups, lr), moms=(0.8, 0.9))

In [0]:
learner.save('second_cycle_bert')

In [0]:
seed_all(seed)
learner.load('second_cycle_bert');

In [0]:
learner.freeze_to(-3)

In [0]:
learner.fit_one_cycle(10, max_lr=slice(lr*0.95**num_groups, lr), moms=(0.8, 0.9))

In [0]:
learner.save('third_cycle_bert')

In [0]:
seed_all(seed)
learner.load('third_cycle_bert');

In [0]:
learner.predict('This is the best movie of 2020')

In [0]:
learner.predict('such a bitch')

In [0]:
learner.predict('make me a sandwich good for nothing woman')

LINEAR MODELS

In [0]:
df5 = train_data[['Sub-task B']]

In [0]:
le = preprocessing.LabelEncoder()
df55 = df5.apply(le.fit_transform)
df55.head()

In [0]:
enc = preprocessing.OneHotEncoder()
enc.fit(df55)
onehotlabels2 = enc.transform(df55).toarray()
onehotlabels2.shape

In [0]:
onehotlabels2

In [0]:
df555 = pd.DataFrame(onehotlabels2)

In [0]:
df555.head()

In [0]:
df2 = train_data['Text']

In [0]:
df22 = np.array(df2)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df22, df55['Sub-task B'], random_state = 42)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.transform(X_test)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(models, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

In [0]:
from gensim.models import KeyedVectors
model_ug_cbow = KeyedVectors.load('w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('w2v_model_ug_sg.word2vec')

In [0]:
!pip3 install word2vec

In [0]:
nb = MultinomialNB()
y_score=nb.fit(X_train_tfidf, y_train)

In [0]:
y_pred = nb.predict(X_test_tfidf)

In [0]:
nb.score(y_pred,y_test)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df22, df55['Sub-task B'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.transform(X_test)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [0]:
nb = MultinomialNB()
y_score=nb.fit(X_train_tfidf, y_train)

In [0]:
model.score(y_test,y_pred)

In [0]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df22, df55['Sub-task B'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.transform(X_test)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [0]:
nb = MultinomialNB()
y_score=nb.fit(X_train_tfidf, y_train)

In [0]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

In [0]:
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)
score = classifier.score(X_test_tfidf, y_test)

print("Accuracy:", score)

In [0]:
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)
score = classifier.score(X_test_tfidf, y_test)
print('Accuracy for {} data: {:.4f}'.format(source, score))

In [0]:
input_dim = X_train_tfidf.shape[1]  
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

In [0]:
history1 = model.fit(X_train_tfidf, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test_tfidf, y_test),
                    batch_size=10)

In [0]:
f1_score(y_test, y_pred1, average="macro")

In [0]:
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    clean_tweet_texts.append(tweet_cleaner(train['Text'][i]))
nums = [0,len(test)]
test_tweet_texts = []
for i in range(nums[0],nums[1]):
    test_tweet_texts.append(tweet_cleaner(test['Text'][i])) 
train_clean = pd.DataFrame(clean_tweet_texts,columns=['Text'])
train_clean['label'] = train['Sub-task B']
train_clean['ID'] = train.ID
test_clean = pd.DataFrame(test_tweet_texts,columns=[['Text', 'Sub-task B']])
test_clean['ID'] = test.ID

In [0]:
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_clean['Text'],train_clean['label'])
train_x = train_clean['Text']
valid_x = test_clean['Text']
train_y = train_clean['label']
valid_y = test_clean['Sub-task B']
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [0]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(train_clean['Text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    classifier.fit(feature_vector_train, label)
    predictions = classifier.predict(feature_vector_valid)
    return metrics.f1_score(valid_y,predictions,average='macro')

In [0]:
accuracyORIGINAL = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
accuracyORIGINAL = train_model(svm.LinearSVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM Baseline, WordLevel TFIDF: ", accuracyORIGINAL)

In [0]:
ros = RandomOverSampler(random_state=777)
ros_xtrain_tfidf, ros_train_y = ros.fit_sample(xtrain_tfidf, train_y)
accuracyROS = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("LR ORIGINAL, WordLevel TFIDF: ", accuracyROS)
accuracyROS = train_model(svm.LinearSVC(),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("SVM ROS, WordLevel TFIDF: ", accuracyROS)
accuracyROS = train_model(linear_model.SGDClassifier(loss='hinge'),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("SGDClassifier, WordLevel TFIDF: ", accuracyROS)
accuracyROS = train_model(linear_model.RidgeClassifier(alpha=5.0),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("SGDClassifier, WordLevel TFIDF: ", accuracyROS)
accuracyROS = train_model(linear_model.LogisticRegressionCV(Cs=10),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("SGDClassifier, WordLevel TFIDF: ", accuracyROS)